In [1]:
import pandas as pd, numpy as np
import sys, os
from pprint import pprint
import random
from datetime import datetime, timedelta

In [2]:
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
def print_runtime():
    print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print("-"*19)
print_runtime()

2021-09-10 00:38:35
-------------------


# 1. GearVN Data

In [ ]:
source_path = "./data/Data-GEARVN.csv"
pdf_gearvn = pd.read_csv(source_path)
print(pdf_gearvn.shape)
display(pdf_gearvn.head())
print_runtime()

(9782, 28)


,ID tích hợp,Thời gian,Kho lấy hàng,Thành phố,Quận huyện,Mã sản phẩm,Ngành,Hãng,Mã chung,Mã riêng,...,Ghi chú của CSKH,Trạng thái,Chăm sóc KH,Thiếu,Thieu,DH thiếu,Ngày xác nhận,Thời gian xác nhận,Nguồn đơn hàng,Column1
0,hd211915,07-02-21 21:48,2 HCM-HHT-KD-V.2,Hồ Chí Minh,Quận Gò Vấp,CHU-LOG-G304,CHU,LOGITECH,NaN,NaN,...,thieu chuot,Thành công,Nguyễn Anh Nguyên,-,-,-,07-02-21 21:55,0:02:04,SHOWROOMHHT,NaN
1,hd211915,07-02-21 21:48,2 HCM-HHT-KD-V.2,Hồ Chí Minh,Quận Gò Vấp,KB-COR-K63-WIRELESS-MX-RD,KB,CORSAIR,NaN,NaN,...,thieu chuot,Thành công,Nguyễn Anh Nguyên,-,-,-,07-02-21 21:55,0:00:00,SHOWROOMHHT,NaN
2,hd211916,07-02-21 21:47,8 HCM-CQ-KD,Hồ Chí Minh,Quận 1,HDD-WD-BLUE-1TB,HDD,WD,NaN,NaN,...,NaN,Thành công,Trần Thiện Tý,-,-,-,07-02-21 21:48,0:00:00,SHOWROOMHHT,NaN
3,hd211914,07-02-21 21:42,2 HCM-HHT-KD-V.2,Bình Dương,Thị xã Tân Uyên,TAI-STE-ARCTIS-7-WHITE,TAI,STEELSERIES,NaN,NaN,...,NaN,Hệ thống hủy,Nguyễn Anh Nguyên,-,-,-,07-02-21 21:44,0:00:00,SHOWROOMHHT,NaN
4,hd211914,07-02-21 21:42,2 HCM-HHT-KD-V.2,Bình Dương,Thị xã Tân Uyên,PK-KING-BINH-GIU-NHIET-BLK,PK,KINGSTON,NaN,NaN,...,NaN,Hệ thống hủy,Nguyễn Anh Nguyên,-,-,-,07-02-21 21:44,0:00:00,SHOWROOMHHT,NaN


2021-09-10 00:41:31
-------------------


## Separate into small tables

In [ ]:
pdf_agent = pdf_gearvn[["Chăm sóc KH"]].drop_duplicates()
print(f"Number of agents: {pdf_agent.shape[0]}")
display(pdf_agent.head())

print_runtime()

Number of agents: 22


,Chăm sóc KH
0,Nguyễn Anh Nguyên
2,Trần Thiện Tý
5,Phạm Ngọc Hiếu
9,Trần Anh Vũ
12,Võ Quốc Khánh


2021-09-10 00:41:35
-------------------


In [ ]:
pdf_gearvn.columns

Index(['ID tích hợp', 'Thời gian', 'Kho lấy hàng', 'Thành phố', 'Quận huyện',
       'Mã sản phẩm', 'Ngành', 'Hãng', 'Mã chung', 'Mã riêng', 'Chiết khấu',
       'Giá', 'Giá sau CK', 'Số lượng', 'Giá trị đơn hàng', 'Tiền chiết khấu',
       'Tiền đặt cọc', 'Tổng thu', 'Ghi chú của CSKH', 'Trạng thái',
       'Chăm sóc KH', 'Thiếu', 'Thieu', 'DH thiếu', 'Ngày xác nhận',
       'Thời gian xác nhận', 'Nguồn đơn hàng', 'Column1'],
      dtype='object')

In [ ]:
pdf_prod = pdf_gearvn[["Ngành", "Hãng", "Mã sản phẩm", "Kho lấy hàng", "Giá"]].drop_duplicates()
print(f"Number of products: {pdf_prod.shape[0]}")
display(pdf_prod.head())

print_runtime()

Number of products: 2301


,Ngành,Hãng,Mã sản phẩm,Kho lấy hàng,Giá
0,CHU,LOGITECH,CHU-LOG-G304,2 HCM-HHT-KD-V.2,"790,000"
1,KB,CORSAIR,KB-COR-K63-WIRELESS-MX-RD,2 HCM-HHT-KD-V.2,"2,490,000"
2,HDD,WD,HDD-WD-BLUE-1TB,8 HCM-CQ-KD,"980,000"
3,TAI,STEELSERIES,TAI-STE-ARCTIS-7-WHITE,2 HCM-HHT-KD-V.2,"3,990,000"
4,PK,KINGSTON,PK-KING-BINH-GIU-NHIET-BLK,2 HCM-HHT-KD-V.2,-


2021-09-10 00:41:42
-------------------


In [ ]:
# Product Code:
pdf_prod["Ngành"].unique()

array(['CHU', 'KB', 'HDD', 'TAI', 'PK', 'MIC', 'CAM', 'TCG', 'VGA', 'PW',
       'VO', 'CPU', 'RAM', 'SSD', 'MB', 'FAN', 'TBM', 'MAN', 'LC', 'LAP',
       'TUI', 'BALO', 'GHE', 'CT', 'TN', 'LOA', 'BAN', 'PM', 'TAN',
       'CONS', 'PC', 'tai', 'PRINTER', 'BUNDLE'], dtype=object)

In [ ]:
pdf_order = pdf_gearvn[[
    "ID tích hợp", 
    "Thời gian", # Sales created order
    "Giá trị đơn hàng", "Tiền chiết khấu", "Tiền đặt cọc", "Tổng thu",
    "Ghi chú của CSKH", "Trạng thái", "Chăm sóc KH", "Thiếu", "Thieu", "DH thiếu", 
    "Ngày xác nhận", # Storage confirmed timestamp
    "Nguồn đơn hàng",
]].fillna("0").groupby(["ID tích hợp"], as_index=False).agg({
    "Thời gian": "max", 
    "Giá trị đơn hàng": "max", 
    "Tiền chiết khấu": "max", 
    "Tiền đặt cọc": "max", 
    "Tổng thu": "max",
    "Ghi chú của CSKH": "max", 
    "Trạng thái": "max", 
    "Chăm sóc KH": "max", 
    "Thiếu": "max", 
    "Thieu": "max", 
    "DH thiếu": "max", 
    "Ngày xác nhận": "max", 
    "Nguồn đơn hàng": "max"
})

print(f"Number of order: {pdf_order.shape[0]}")
display(pdf_order.head())

print_runtime()

Number of order: 3400


,ID tích hợp,Thời gian,Giá trị đơn hàng,Tiền chiết khấu,Tiền đặt cọc,Tổng thu,Ghi chú của CSKH,Trạng thái,Chăm sóc KH,Thiếu,Thieu,DH thiếu,Ngày xác nhận,Nguồn đơn hàng
0,hd108850-9,05-02-21 19:46,"294,000,000",0,0,"294,000,000",0,Hệ thống hủy,Nguyễn Anh Tuấn,1,-,1,06-02-21 7:58,OTHER
1,hd170656-6,06-02-21 18:11,"193,500,000","1,270,000",0,"193,500,000",0,Hệ thống hủy,Nguyễn Thế Vinh,-,-,-,06-02-21 18:17,OTHER
2,hd203577-1,01-02-21 8:11,"17,830,000",0,0,"17,830,000",0,Thành công,Nguyễn Thế Nam,-,-,-,02-02-21 12:52,SHOWROOMHHT
3,hd203879-1,01-02-21 8:43,0,"9,000,000",0,0,0,Thành công,Lê Quang Hoàng Dũng,-,-,-,02-04-21 9:05,SHOWROOMHHT
4,hd203913-2,05-02-21 18:41,"147,250,003","4,329,997",0,"147,250,003",0,Hệ thống hủy,0,-,-,-,0,OTHER


2021-09-10 00:41:55
-------------------


In [ ]:
pdf_confirm_order = pdf_order[pdf_gearvn["Trạng thái"]=="Thành công"]
print(pdf_confirm_order.shape)
print_runtime()

(2605, 14)
2021-09-10 00:43:55
-------------------


/Users/newuser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
confirm_rate = pdf_confirm_order.shape[0]/pdf_order.shape[0]
print(f"Confirm rated: {confirm_rate}")
print_runtime()

Confirm rated: 0.7661764705882353
2021-09-10 00:45:01
-------------------


# TODO:
- pdf_order_detail
- Add column to show online and offline orders 

# TODO: Prepare logic of generating data for below reports:
- Ratio of online and offline over total revenue. From Showroom's Sales are Offline.
- Ratio of Gaming Gears included in full PC and separated 
